In [21]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os 

In [26]:
# Initialize the master DataFrame
master_df = pd.read_csv("ZIPdf.csv")
master_df.rename(columns={"Unnamed: 0" : "ZIP"}, inplace=True)
master_df["ZIP"] = master_df["ZIP"].astype(str).str.zfill(5)
master_df.set_index("ZIP", inplace=True)
master_df

,Num Farms,Num Estabs,Total Emp Bus,Min Share,Black Share,Number of Loans_1,Total Loans Amount_1,Total Employees_1,Number of Loans_2,Total Loans Amount_2,Total Employees_2,Number of Loans_3,Total Loans Amount_3,Total Employees_3
ZIP,,,,,,,,,,,,,,
00501,0.0,10.0,10.0,NaN,NaN,0,0,0,0,0,0,0,0,0
01001,11.0,475.0,486.0,0.087898,0.028075,0,0,0,0,0,0,0,0,0
01002,60.0,502.0,562.0,0.257683,0.058108,0,0,0,0,0,0,0,0,0
01003,3.0,15.0,18.0,0.311443,0.044960,0,0,0,0,0,0,0,0,0
01004,3.0,6.0,9.0,NaN,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99926,1.0,9.0,10.0,0.890013,0.000000,0,0,0,0,0,0,0,0,0
99927,0.0,0.0,0.0,NaN,NaN,0,0,0,0,0,0,0,0,0
99928,1.0,4.0,5.0,NaN,NaN,0,0,0,0,0,0,0,0,0


In [27]:
# List of file names
base = "C:\\Users\\hthog\\Desktop\\PPP Loan Data\\"
file_df = pd.read_csv(base + "Files.csv")
file_df

,File,Completed
0,Test.csv,0
1,public_150k_plus_230630.csv,0
2,public_up_to_150k_1_230630.csv,0
3,public_up_to_150k_2_230630.csv,0
4,public_up_to_150k_3_230630.csv,0
5,public_up_to_150k_4_230630.csv,0
6,public_up_to_150k_5_230630.csv,0
7,public_up_to_150k_6_230630.csv,0
8,public_up_to_150k_7_230630.csv,0
9,public_up_to_150k_8_230630.csv,0


In [28]:
# Define the processing function
def process(row, p):
    # If not a non-profit
    if not row[50]:
        print(type(row[1]))
        print(type(datetime.strptime(row[1],'%m/%d/%Y')))
        print(datetime.strptime(row[1],'%m/%d/%Y'))
#     if not row[8]:
#         p.write(row)
#     else:
#         zip_code = row[8].astype(str)[:5]
#         data_row = master_df.loc[zip_code]


In [31]:
for file in file_df[file_df["Completed"] == 0]["File"]:
    file_name = base + file
    # Check if file exists
    if not os.path.exists(file_name):
        print(f"File {file_name} does not exist. Skipping...")
        continue

    print(f"Processing {file}...")
    
    # Read file row by row since cannot fit in DF
    with open("Problem Rows.csv", "a") as p:
        chunk_size = 50000
        for chunk in pd.read_csv(file_name, chunksize=chunk_size):
            chunk = chunk[chunk["NonProfit"] != "Y"]
            chunk = chunk[~chunk["BusinessType"].isin(["Sole Proprietorship",
                                                      "Independent Contractors",
                                                      "Self-Employed Individuals"])]
            chunk = chunk[~chunk["NAICS"].isin([999990, None])]
        
#         with open(file_name, 'r') as f:
#             reader = csv.reader(f)
#             headers = next(reader)
#             for row in reader:
#                 process(row, p)

    #file_df[file_df["File"] == file]["Completed"] = 1
    
    print(f"Updated master DataFrame with data from {file}.")
    
    # Ask for user confirmation to proceed to the next file
    user_input = input("Do you want to continue with the next file? (y/n): ")
    if user_input.lower() != 'y':
        print("Stopping the process.")
        # Save the master DataFrame to disk
        master_df.to_csv('ZIPdf.csv')
        file_df.to_csv(base + "Files.csv", index=False)
        break

Processing Test.csv...
   LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  9547507704     5/1/2020            464              PPP   
1  9777677704     5/1/2020            464              PPP   
2  5791407702     5/1/2020           1013              PPP   

              BorrowerName        BorrowerAddress      BorrowerCity  \
0    SUMTER COATINGS, INC.  2410 Highway 15 South            Sumter   
1    PLEASANT PLACES, INC.    7684 Southrail Road  North Charleston   
2  BOYER CHILDREN'S CLINIC       1850 BOYER AVE E           SEATTLE   

   BorrowerState BorrowerZip LoanStatusDate  ...             BusinessType  \
0            NaN  29150-9662     12/18/2020  ...              Corporation   
1            NaN  29420-9000      9/28/2021  ...      Sole Proprietorship   
2            NaN  98112-2922      3/17/2021  ...  Non-Profit Organization   

   OriginatingLenderLocationID                      OriginatingLender  \
0                        19248                           Syno